In [4]:
import yfinance as yf
import pandas as pd
from ta import add_all_ta_features
from ta.trend import SMAIndicator, EMAIndicator

In [5]:
# Fetch the data from Yahoo Finance and return it as a dataframe
def fetch_ohlcv_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data.reset_index(inplace=True)
    return data

In [10]:
ticker = 'IOO.AX'  # SharesGlobal 100 ETF (IOO) listed on the Australian market
start_date = '2021-01-01'
end_date = '2023-05-04'

# Fetch the data
data = fetch_ohlcv_data(ticker, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [11]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,83.000000,83.000000,81.550003,81.839996,81.839996,27405
1,2021-01-05,81.690002,81.690002,80.949997,81.040001,81.040001,28391
2,2021-01-06,81.099998,81.330002,80.400002,80.449997,80.449997,37254
3,2021-01-07,80.809998,81.800003,80.629997,81.070000,81.070000,42802
4,2021-01-08,81.510002,82.470001,81.510002,82.279999,82.279999,8752
...,...,...,...,...,...,...,...
583,2023-04-27,107.900002,107.900002,107.110001,107.279999,107.279999,16685
584,2023-04-28,108.820000,109.160004,108.629997,109.160004,109.160004,25948
585,2023-05-01,109.360001,109.360001,108.940002,109.010002,109.010002,31654
586,2023-05-02,108.980003,108.980003,107.389999,107.389999,107.389999,31082


In [29]:
# Add SMA and EMA for the close price with a "window" days window
def add_sma(data, column_name, window):
    sma_indicator = SMAIndicator(data[column_name], window=window)
    data[f'sma_{window}'] = sma_indicator.sma_indicator()
    return data

def add_ema(data, column_name, window):
    ema_indicator = EMAIndicator(data[column_name], window=window)
    data[f'ema_{window}'] = ema_indicator.ema_indicator()
    return data

In [30]:
# Add SMA and EMA for the close price with certain window
window_list = [5, 10, 20, 50, 100, 200]
for window_size in window_list:
    data = add_sma(data, 'Close', window_size)
    data = add_ema(data, 'Close', window_size)

In [31]:
# Add all ta features
data = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume")

/home/gerry/anaconda3/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/gerry/anaconda3/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [32]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sma_5,ema_5,sma_10,...,sma_10_signal,ema_10_signal,sma_20_signal,ema_20_signal,sma_50_signal,ema_50_signal,sma_100_signal,ema_100_signal,Buy_count,Sell_count
0,2021-01-04,83.000000,83.000000,81.550003,81.839996,81.839996,27405,NaN,NaN,NaN,...,Sell,Sell,Sell,Sell,Sell,Sell,Sell,Sell,0,10
1,2021-01-05,81.690002,81.690002,80.949997,81.040001,81.040001,28391,NaN,NaN,NaN,...,Sell,Sell,Sell,Sell,Sell,Sell,Sell,Sell,0,10
2,2021-01-06,81.099998,81.330002,80.400002,80.449997,80.449997,37254,NaN,NaN,NaN,...,Sell,Sell,Sell,Sell,Sell,Sell,Sell,Sell,0,10
3,2021-01-07,80.809998,81.800003,80.629997,81.070000,81.070000,42802,NaN,NaN,NaN,...,Sell,Sell,Sell,Sell,Sell,Sell,Sell,Sell,0,10
4,2021-01-08,81.510002,82.470001,81.510002,82.279999,82.279999,8752,81.335999,81.530616,NaN,...,Sell,Sell,Sell,Sell,Sell,Sell,Sell,Sell,2,8


In [33]:
data[-5:].loc[:, ['Date','Close','Volume','sma_5','ema_5',
                  'sma_10','ema_10','sma_20','ema_20','sma_50','ema_50','sma_100','ema_100']]

,Date,Close,Volume,sma_5,ema_5,sma_10,ema_10,sma_20,ema_20,sma_50,ema_50,sma_100,ema_100
583,2023-04-27,107.279999,16685,106.806000,106.899029,106.127000,106.180879,104.7445,104.833536,101.3100,102.108234,98.7194,100.100868
584,2023-04-28,109.160004,25948,107.398001,107.652687,106.588000,106.722538,105.1545,105.245580,101.5244,102.384774,98.8300,100.280257
585,2023-05-01,109.010002,31654,107.992001,108.105126,106.981001,107.138441,105.5555,105.604097,101.7336,102.644587,98.9257,100.453123
586,2023-05-02,107.389999,31082,108.178001,107.866750,107.098000,107.184179,105.8135,105.774183,101.8896,102.830681,99.0085,100.590487
587,2023-05-03,107.680000,35506,108.104001,107.804500,107.302000,107.274328,106.0570,105.955689,102.0674,103.020851,99.0983,100.730873


In [34]:
def get_signal(data, column_price, column_indicator):
    if data[column_price] > data[column_indicator]:
        return "Buy"
    else:
        return "Sell"

def add_signals(data, column_price, window_list):
    for window_size in window_list:
        sma_column = f'sma_{window_size}'
        ema_column = f'ema_{window_size}'
        data[f'{sma_column}_signal'] = data.apply(get_signal, args=(column_price, sma_column), axis=1)
        data[f'{ema_column}_signal'] = data.apply(get_signal, args=(column_price, ema_column), axis=1)
    return data

def count_signals(data, signal_type, window_list):
    signal_columns = [f'{indicator}_{window_size}_signal' for indicator in ['sma', 'ema'] for window_size in window_list]
    return data[signal_columns].apply(lambda x: (x == signal_type).sum(), axis=1)

In [35]:
window_list = [5, 10, 20, 50, 100, 200]
data_with_signals = add_signals(data, 'Close', window_list)
data_with_signals['Buy_count'] = count_signals(data_with_signals, 'Buy', window_list)
data_with_signals['Sell_count'] = count_signals(data_with_signals, 'Sell', window_list)

In [36]:
data_with_signals[-5:].loc[:, ['Date','Close','Volume',
                               'sma_5','ema_5','sma_10','ema_10','sma_20','ema_20',
                               'sma_50','ema_50','sma_100','ema_100']]

,Date,Open,High,Low,Close,Adj Close,Volume,sma_5,ema_5,sma_10,...,sma_20_signal,ema_20_signal,sma_50_signal,ema_50_signal,sma_100_signal,ema_100_signal,Buy_count,Sell_count,sma_200_signal,ema_200_signal
0,2021-01-04,83.000000,83.000000,81.550003,81.839996,81.839996,27405,NaN,NaN,NaN,...,Sell,Sell,Sell,Sell,Sell,Sell,0,12,Sell,Sell
1,2021-01-05,81.690002,81.690002,80.949997,81.040001,81.040001,28391,NaN,NaN,NaN,...,Sell,Sell,Sell,Sell,Sell,Sell,0,12,Sell,Sell
2,2021-01-06,81.099998,81.330002,80.400002,80.449997,80.449997,37254,NaN,NaN,NaN,...,Sell,Sell,Sell,Sell,Sell,Sell,0,12,Sell,Sell
3,2021-01-07,80.809998,81.800003,80.629997,81.070000,81.070000,42802,NaN,NaN,NaN,...,Sell,Sell,Sell,Sell,Sell,Sell,0,12,Sell,Sell
4,2021-01-08,81.510002,82.470001,81.510002,82.279999,82.279999,8752,81.335999,81.530616,NaN,...,Sell,Sell,Sell,Sell,Sell,Sell,2,10,Sell,Sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,2023-04-27,107.900002,107.900002,107.110001,107.279999,107.279999,16685,106.806000,106.899029,106.127000,...,Buy,Buy,Buy,Buy,Buy,Buy,12,0,Buy,Buy
584,2023-04-28,108.820000,109.160004,108.629997,109.160004,109.160004,25948,107.398001,107.652687,106.588000,...,Buy,Buy,Buy,Buy,Buy,Buy,12,0,Buy,Buy
585,2023-05-01,109.360001,109.360001,108.940002,109.010002,109.010002,31654,107.992001,108.105126,106.981001,...,Buy,Buy,Buy,Buy,Buy,Buy,12,0,Buy,Buy
586,2023-05-02,108.980003,108.980003,107.389999,107.389999,107.389999,31082,108.178001,107.866750,107.098000,...,Buy,Buy,Buy,Buy,Buy,Buy,10,2,Buy,Buy


In [37]:
def generate_summary_table(data, current_price, window_list):
    columns = ['Period', 'Simple', 'Exponential']
    summary_table = pd.DataFrame(columns=columns)

    for window_size in window_list:
        sma_signal = get_signal({'Close': current_price,
                                 'sma_{}'.format(window_size): data['sma_{}'.format(window_size)].iloc[-1]}, 
                                'Close', 'sma_{}'.format(window_size))
        ema_signal = get_signal({'Close': current_price, 
                                 'ema_{}'.format(window_size): data['ema_{}'.format(window_size)].iloc[-1]}, 
                                'Close', 'ema_{}'.format(window_size))
        summary_table = summary_table.append(pd.Series([f'MA{window_size}', 
                                                        f'{data["sma_{}".format(window_size)].iloc[-1]:.2f} {sma_signal}', 
                                                        f'{data["ema_{}".format(window_size)].iloc[-1]:.2f} {ema_signal}'], 
                                                       index=columns), ignore_index=True)

    buy_count = (summary_table['Simple'].apply(lambda x: x.split()[1]) == 'Buy').sum() + (summary_table['Exponential'].apply(lambda x: x.split()[1]) == 'Buy').sum()
    sell_count = (summary_table['Simple'].apply(lambda x: x.split()[1]) == 'Sell').sum() + (summary_table['Exponential'].apply(lambda x: x.split()[1]) == 'Sell').sum()
    summary = 'BUY' if buy_count > sell_count else 'SELL'

    summary_table = summary_table.append(pd.Series(['Buy:', buy_count, f'Sell: {sell_count}'], index=columns), ignore_index=True)
    summary_table = summary_table.append(pd.Series(['Summary:', '', summary], index=columns), ignore_index=True)

    return summary_table

current_price = 106.92
summary_table = generate_summary_table(data_with_signals, current_price, window_list)
print(summary_table)

     Period       Simple  Exponential
0       MA5  108.10 Sell  107.80 Sell
1      MA10  107.30 Sell  107.27 Sell
2      MA20   106.06 Buy   105.96 Buy
3      MA50   102.07 Buy   103.02 Buy
4     MA100    99.10 Buy   100.73 Buy
5     MA200    98.16 Buy    99.25 Buy
6      Buy:            8      Sell: 4
7  Summary:                       BUY


/tmp/ipykernel_197500/3838712495.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_table = summary_table.append(pd.Series([f'MA{window_size}', f'{data["sma_{}".format(window_size)].iloc[-1]:.2f} {sma_signal}', f'{data["ema_{}".format(window_size)].iloc[-1]:.2f} {ema_signal}'], index=columns), ignore_index=True)
/tmp/ipykernel_197500/3838712495.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_table = summary_table.append(pd.Series([f'MA{window_size}', f'{data["sma_{}".format(window_size)].iloc[-1]:.2f} {sma_signal}', f'{data["ema_{}".format(window_size)].iloc[-1]:.2f} {ema_signal}'], index=columns), ignore_index=True)
/tmp/ipykernel_197500/3838712495.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  su

In [38]:
summary_table

,Period,Simple,Exponential
0,MA5,108.10 Sell,107.80 Sell
1,MA10,107.30 Sell,107.27 Sell
2,MA20,106.06 Buy,105.96 Buy
3,MA50,102.07 Buy,103.02 Buy
4,MA100,99.10 Buy,100.73 Buy
5,MA200,98.16 Buy,99.25 Buy
6,Buy:,8,Sell: 4
7,Summary:,,BUY
